# 03. Feature Extraction

In [10]:
# 01. CONFIGURAÇÃO E EPOCAGEM

import os
import mne
import warnings
import numpy as np
import pandas as pd

mne.set_log_level('WARNING')
warnings.filterwarnings('ignore')

# 1. Diretórios
DIR_CLEANED = '../data/interim/cleaned_data/'
DIR_EPOCHS = '../data/processed/epochs/'
DIR_REPORTS = '../reports/'

for d in [DIR_EPOCHS, DIR_REPORTS]:
    os.makedirs(d, exist_ok=True)

# 2. Dicionário de Marcações
EVENT_DICT = {
    'FF': 1, 'F': 1,  # Face Feliz
    'FN': 2, 'N': 2,  # Face Neutra
    'FR': 3, 'R': 3   # Face Raiva
}

# 3. Parâmetros das Frequências
BANDAS_FREQ = {
    'Theta': (4.0, 8.0),
    'Alpha': (8.0, 13.0),
    'Beta': (13.0, 30.0),
    'Gamma': (30.0, 40.0)
}

CANAIS_PSD = ['F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2']
CANAIS_N170 = ['O1', 'O2']
JANELA_N170 = (0.130, 0.230)

arquivos_limpos = sorted([f for f in os.listdir(DIR_CLEANED) if f.endswith('.fif')])
print(f"Iniciando epocagem para {len(arquivos_limpos)} blocos...")

# 4. Segmentação (Epocagem)
for arquivo in arquivos_limpos:
    paciente_id = arquivo.replace('-cleaned-raw.fif', '')
    caminho_entrada = os.path.join(DIR_CLEANED, arquivo)
    caminho_saida = os.path.join(DIR_EPOCHS, f"{paciente_id}-epo.fif")
          
    try:
        raw = mne.io.read_raw_fif(caminho_entrada, preload=True, verbose=False)
        
        if 'faces' in arquivo:
            events, event_id_encontrados = mne.events_from_annotations(
                raw, 
                event_id=EVENT_DICT, 
                regexp='^(FF|FN|FR|F|N|R)$', 
                verbose=False
            )
            
            if len(events) > 0:
                epochs = mne.Epochs(raw, events, event_id=event_id_encontrados, 
                                    tmin=-0.2, tmax=0.8, baseline=(None, 0), preload=True, verbose=False)
                epochs.save(caminho_saida, overwrite=True, verbose=False)
                print(f"[{paciente_id}] Sucesso ({len(epochs)} épocas)")
            else:
                print(f"[{paciente_id}] Nenhuma marcação de face encontrada.")
                
        elif 'basal' in arquivo:
            if raw.times[-1] > 3.0: 
                raw.crop(tmin=1.0)
            epochs = mne.make_fixed_length_epochs(raw, duration=2.0, preload=True, verbose=False)
            epochs.save(caminho_saida, overwrite=True, verbose=False)
            print(f"[{paciente_id}] Sucesso (Basal)")

    except Exception as e:
        print(f"Erro no processamento de {arquivo}: {e}")

print("\nEpocagem finalizada.")

Iniciando epocagem para 126 blocos...
[TEA_G_26_basal_pos] Sucesso (Basal)
[TEA_G_26_basal_pre] Sucesso (Basal)
[TEA_G_26_faces] Sucesso (30 épocas)
[TEA_G_28_basal_pos] Sucesso (Basal)
[TEA_G_28_basal_pre] Sucesso (Basal)
[TEA_G_28_faces] Sucesso (30 épocas)
[TEA_G_30_basal_pos] Sucesso (Basal)
[TEA_G_30_basal_pre] Sucesso (Basal)
[TEA_G_30_faces] Sucesso (30 épocas)
[TEA_G_40_basal_pos] Sucesso (Basal)
[TEA_G_40_basal_pre] Sucesso (Basal)
[TEA_G_40_faces] Sucesso (30 épocas)
[TEA_L_16_basal_pos] Sucesso (Basal)
[TEA_L_16_basal_pre] Sucesso (Basal)
[TEA_L_16_faces] Sucesso (30 épocas)
[TEA_L_17_basal_pos] Sucesso (Basal)
[TEA_L_17_basal_pre] Sucesso (Basal)
[TEA_L_17_faces] Sucesso (30 épocas)
[TEA_L_24_basal_pos] Sucesso (Basal)
[TEA_L_24_basal_pre] Sucesso (Basal)
[TEA_L_24_faces] Sucesso (30 épocas)
[TEA_L_25_basal_pos] Sucesso (Basal)
[TEA_L_25_basal_pre] Sucesso (Basal)
[TEA_L_25_faces] Sucesso (27 épocas)
[TEA_L_27_basal_pos] Sucesso (Basal)
[TEA_L_27_basal_pre] Sucesso (Basal)


In [11]:
# 02. EXTRAÇÃO DE FEATURES

import glob
from scipy.stats import entropy

print("\n" + "="*80)
print(" Iniciando Extração...")
print("="*80)

ARQUIVO_SAIDA = os.path.join(DIR_REPORTS, 'tabela_features_eeg_completa.csv')
PARES_ASSIMETRIA = [('F4', 'F3'), ('C4', 'C3'), ('P4', 'P3'), ('O2', 'O1')]
PARES_CONECTIVIDADE = [('F3', 'F4'), ('C3', 'C4'), ('P3', 'P4'), ('O1', 'O2'), ('F3', 'O1'), ('F4', 'O2')]

arquivos_epo = glob.glob(os.path.join(DIR_EPOCHS, '*-epo.fif'))
todas_features = []

for caminho_arquivo in arquivos_epo:
    nome_arquivo = os.path.basename(caminho_arquivo)
    sujeito_id = nome_arquivo.split('-epo')[0]
    grupo = 'TEA' if 'TEA' in sujeito_id else 'Control'
    
    epochs = mne.read_epochs(caminho_arquivo, preload=True, verbose=False)
    
    for condicao in epochs.event_id.keys():
        epochs_condicao = epochs[condicao]
        if len(epochs_condicao) == 0:
            continue

        # A) PSD (Welch)
        # n_fft adaptado ao tamanho real da época para evitar erros matemático    
        n_tempos = epochs_condicao.get_data().shape[2]
        n_fft = min(256, n_tempos) 
        
        psd_data = epochs_condicao.compute_psd(method='welch', fmin=4.0, fmax=40.0, 
                                               n_fft=n_fft, n_overlap=n_fft//2, verbose=False)
        psds, freqs = psd_data.get_data(return_freqs=True)
        psds_media = np.mean(psds, axis=0) 
        
        # B) Normalização dos nomes das condições para o Aprendizado de Máquina
        nome_condicao_limpo = condicao
        if condicao in ['FF', 'F']: nome_condicao_limpo = 'Face Feliz'
        elif condicao in ['FN', 'N']: nome_condicao_limpo = 'Face Neutra'
        elif condicao in ['FR', 'R']: nome_condicao_limpo = 'Face Raiva'
        
        features_linha = {
            'ID': sujeito_id,
            'Grupo': grupo,
            'Condicao': nome_condicao_limpo,
            'Tipo': 'Task' if 'Face' in nome_condicao_limpo else 'Resting'
        }
        
        # C. Potência Relativa, TBR e Entropia Espectral
        potencia_absoluta = {}
        for ch_idx, canal in enumerate(epochs_condicao.ch_names):
            if canal not in CANAIS_PSD: continue
            potencia_total_canal = 0
            
            # 1. Potência Absoluta
            for banda, (fmin, fmax) in BANDAS_FREQ.items():
                idx_banda = np.logical_and(freqs >= fmin, freqs <= fmax)
                potencia = np.sum(psds_media[ch_idx, idx_banda])
                potencia_absoluta[f"{banda}_{canal}"] = potencia
                potencia_total_canal += potencia
            
            # 2. Potência Relativa
            for banda in BANDAS_FREQ.keys():
                abs_val = potencia_absoluta[f"{banda}_{canal}"]
                rel_val = abs_val / potencia_total_canal if potencia_total_canal > 0 else 0
                features_linha[f"{banda}Rel_{canal}"] = rel_val
            
            # 3. TBR (Indicador de desatenção/controle inibitório) 
            theta_val = potencia_absoluta[f"Theta_{canal}"]
            beta_val = potencia_absoluta[f"Beta_{canal}"]
            features_linha[f"TBR_{canal}"] = theta_val / beta_val if beta_val > 0 else 0
            
            # 4. Entropia Espectral
            psd_norm = psds_media[ch_idx, :] / np.sum(psds_media[ch_idx, :])
            features_linha[f"Entropia_{canal}"] = entropy(psd_norm)

        # D. Assimetria Hemisférica 
        for banda in BANDAS_FREQ.keys():
            for ch_dir, ch_esq in PARES_ASSIMETRIA:
                chave_dir = f"{banda}Rel_{ch_dir}"
                chave_esq = f"{banda}Rel_{ch_esq}"
                
                if chave_dir in features_linha and chave_esq in features_linha:
                    nome_regiao = ch_dir[0] 
                    features_linha[f"Asym_{banda}_{nome_regiao}"] = features_linha[chave_dir] - features_linha[chave_esq]

        # E. Conectividade Funcional
        dados_evoked = np.mean(epochs_condicao.get_data(), axis=0)
        
        for ch1, ch2 in PARES_CONECTIVIDADE:
            if ch1 in epochs_condicao.ch_names and ch2 in epochs_condicao.ch_names:
                idx1 = epochs_condicao.ch_names.index(ch1)
                idx2 = epochs_condicao.ch_names.index(ch2)
                corr = np.corrcoef(dados_evoked[idx1, :], dados_evoked[idx2, :])[0, 1]
                features_linha[f"Conn_{ch1}_{ch2}"] = corr

        todas_features.append(features_linha)

df_features = pd.DataFrame(todas_features)
df_features.to_csv(ARQUIVO_SAIDA, index=False)

print(f"Matriz de características gerada com sucesso.")
print(f"Dimensões do dataset de saída: {df_features.shape[0]} amostras x {df_features.shape[1]} atributos.")
print("="*80)


 Iniciando Extração...
Matriz de características gerada com sucesso.
Dimensões do dataset de saída: 213 amostras x 74 atributos.
